## Part 1 - 自动并行计算

In [1]:
import torch
import time

assert torch.cuda.device_count() >= 2


AssertionError: 

In [2]:
class Benchmark():  # 本类已保存在d2lzh_pytorch包中方便以后使用
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

        
def run(x):
    for _ in range(20000):
        y = torch.mm(x, x)

x_gpu1 = torch.rand(size=(100, 100), device='cuda:0')
x_gpu2 = torch.rand(size=(100, 100), device='cuda:1')


with Benchmark('Run on GPU1.'):
    run(x_gpu1)
    torch.cuda.synchronize()

with Benchmark('Then run on GPU2.'):
    run(x_gpu2)
    torch.cuda.synchronize()

RuntimeError: CUDA error: invalid device ordinal

In [3]:
with Benchmark('Run on both GPU1 and GPU2 in parallel.'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

Run on both GPU1 and GPU2 in parallel. time: 0.4170 sec


NameError: name 'x_gpu2' is not defined

## Part 2 - 多GPU计算

In [4]:
import torch
net = torch.nn.Linear(10, 1).cuda()
net

Linear(in_features=10, out_features=1, bias=True)

In [5]:
net = torch.nn.DataParallel(net)
net


DataParallel(
  (module): Linear(in_features=10, out_features=1, bias=True)
)